# Response format - Example

In [8]:
import os
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional, List

## 初始化客户端

In [9]:
# 这里使用 OpenAI SDK 调用 智谱AI-API
os.environ["OPENAI_API_KEY"] = "*******************"
os.environ["OPENAI_BASE_URL"] = "https://open.bigmodel.cn/api/paas/v4/"

client = OpenAI(
    # api_key="YOUR_API_KEY",
    # base_url="BASE_URL"
)

## 生成引导提示词

`@format_prompt_getter`：装饰在BaseModel上，能够为Model提供两个方法，分别是：

In [10]:
from response_format import format_prompt_getter

class Book(BaseModel):
    title: str = Field(description="书名")
    author: Optional[str] = Field(description="书的作者")

@format_prompt_getter
class FamoursBook(BaseModel):
    books: List[Book] = Field(description="书籍列表")

- `get_response_format_prompt`：解析模型结构和其字段信息（包括每个子模型的字段信息），获取对应的引导提示词和JSON样例，提示模型输出正确的 JSON 格式。

In [11]:
print(f"get_response_format_prompt: \n{FamoursBook.get_response_format_prompt()}")

get_response_format_prompt: 
Kindly provide the response if you have the answer and output them in JSON format.

EXAMPLE JSON OUTPUT AND DESCRIPTIONS:
{
    "books": [
        {
            "title": "书名",
            "author": "书的作者"
        },
        (... more books.书籍列表 ...)
    ]
}



- `get_json_output_prompt`: 相比`get_response_format_prompt`方法，该方法仅给出了希望模型输出的 JSON 格式的样例，本质上是也是一个提示词，你可以在此基础上增加自定义的引导提示词内容。
- 对于可迭代类型的字段，该方法会额外增加提示词`(... more books.{book.field_info.description} ...)`以描述可迭代对象，除此之外不包含额外的提示语内容。

In [12]:
print(f"get_json_output_prompt: \n{FamoursBook.get_json_output_prompt()}")

get_json_output_prompt: 
{
    "books": [
        {
            "title": "书名",
            "author": "书的作者"
        },
        (... more books.书籍列表 ...)
    ]
}


如果你不喜欢在JSON样例中出现类似``(... more ***.*** ...)``等提示词，可以使用`parse_model_to_json`方法将模型解析为JSON文本和JSON对象。

In [13]:
from response_format import parse_model_to_json

json_text, json_obj = parse_model_to_json(FamoursBook)

print(f"model_to_json_text: \n{json_text}")
print(f"model_to_json_obj: \n{json_obj}")

model_to_json_text: 
{
  "books": [
    {
      "title": "书名",
      "author": "书的作者"
    }
  ]
}
model_to_json_obj: 
{'books': [{'title': '书名', 'author': '书的作者'}]}


## 通过提示使模型以结构化输出

这里通过`FamoursBook.get_response_format_prompt()`获取模型`FamoursBook`对应的Response-Format Prompt（引导提示词和JSON样例），并作为 System Prompt 插入到对话中。
传入的 `system` 的提示词更有利于让模型严格按照希望的 JSON 格式来输出，以实现输出的结构化，便于后续逻辑进行解析。

### 注意事项
1. 用户传入的 `system` 或 `user` 的 prompt 中必须含有 json 字样，并给出希望模型输出的 JSON 格式的样例，以指导模型来输出合法 JSON。
2. 需要合理设置 `max_tokens` 参数(足够长)，防止 JSON 字符串被中途截断。

In [14]:
user_prompt = """中国的四大名著是指那几本书？"""
response = client.chat.completions.create(
    model="glm-4-flash",
    messages=[
        {"role": "system", "content": f"{FamoursBook.get_response_format_prompt()}"},
        {"role": "user", "content": f"{user_prompt}"}
    ],
    top_p=0.7,
    temperature=0.1,
    max_tokens=4000
)

## 解析模型输出结果

In [15]:
print(f"client: \n{response.choices[0].message.content}")

client: 
```json
{
    "books": [
        {
            "title": "红楼梦",
            "author": "曹雪芹"
        },
        {
            "title": "西游记",
            "author": "吴承恩"
        },
        {
            "title": "三国演义",
            "author": "罗贯中"
        },
        {
            "title": "水浒传",
            "author": "施耐庵"
        }
    ]
}
```


`parse_content_to_json`：解析大模型的输出内容，截取` ```json ` 和 ` ``` `中的文本内容，并转换为JSON字符串和JSON对象。

In [16]:
from response_format import parse_content_to_json

json_text, json_object = parse_content_to_json(
    response.choices[0].message.content
)
print(f"json_text: \n{json_text}")
print(f"json_object: \n{json_object}")

json_text: 
{
    "books": [
        {
            "title": "红楼梦",
            "author": "曹雪芹"
        },
        {
            "title": "西游记",
            "author": "吴承恩"
        },
        {
            "title": "三国演义",
            "author": "罗贯中"
        },
        {
            "title": "水浒传",
            "author": "施耐庵"
        }
    ]
}
json_object: 
{'books': [{'title': '红楼梦', 'author': '曹雪芹'}, {'title': '西游记', 'author': '吴承恩'}, {'title': '三国演义', 'author': '罗贯中'}, {'title': '水浒传', 'author': '施耐庵'}]}


# 利用JSON对象实例化模型对象

利用`BaseModel`提供的`model_validate`方法，能够将前面解析得到的JSON对象实例化为模型对象。

In [17]:
for book in FamoursBook.model_validate(json_object).books:
    print(f"书名: {book.title} —— 作者：{book.author}")

书名: 红楼梦 —— 作者：曹雪芹
书名: 西游记 —— 作者：吴承恩
书名: 三国演义 —— 作者：罗贯中
书名: 水浒传 —— 作者：施耐庵
